# Recommender Systems Using Collaborative Filtering

> *Recommender Systems*  
> *MSc in Data Science, Department of Informatics*  
> *Athens University of Economics and Business*

---

Find a ***rating-based*** or ***matching-based*** dataset that can be used to inform a recommender system based on ***collaborative filtering***.

Build a Python notebook that:

- Loads the dataset
- Tries at least 2 different recommendations methods based on collaborative filtering (e.g., Count-based, Matrix Factorization, Tensorflow)
- Uses quantitative metrics to evaluate the recommendations of each of the methods that you selected


## *Table of Contents*

- [*1. Introduction*](#introduction)
    - [*1.1. Libraries*](#libraries)
    - [*1.2. Data*](#data)
    - [*1.3. Data Preprocessing*](#data_preprocessing)
- [*2. Recommendations Using Item-Based Technique*](#item_based_technique)
    - [*2.1. Loading User Ratings*](#loading_data)
    - [*2.2. Create LSH Indices*](#lsh_indices)
    - [*2.3. Make Recommendations*](#item_based_recommendations)
    - [*2.4. Evaluate Recommendations Using Decision Support Methods*](#decision_support_methods)
        - [*2.4.1. Precision*](#precision)
        - [*2.4.2. Recall*](#recall)
    - [*2.5. Evaluate Recommendations Using Ranking Based Methods*](#ranking_based_methods)
        - [*2.5.1. nDCG*](#ndcg)
        - [*2.5.2. Mean Reciprocal Rank*](#mrr)
        - [*2.5.3. Average Precision*](#ap)
- [*3. Recommendations Using Matrix Factorization (SVD)*](#matrix_factorization)
    - [*3.1. Loading Data with Surprise Library*](#loading_data_with_surprise)
    - [*3.2. Selecting Number of Factors*](#n_factors)
    - [*3.3. Train SVD Algorithm*](#train_svd)
    - [*3.4. Make Recommendations*](#svd_recommendations)
    - [*3.5. Evaluate Recommendations*](#svd_evaluation)

---

## Introduction <a class='anchor' id='introduction'></a>

### *Libraries* <a class='anchor' id='libraries'></a>

In [1]:
import numpy as np
import pandas as pd

import time

from surprise import Reader, Dataset, SVD
from surprise.model_selection.search import GridSearchCV

from functions.data_preprocessing import preprocess_ratings_dataset
from functions.data_preprocessing import preprocess_jokes_dataset
from functions.data_preprocessing import unpivot_ratings

from functions.item_based_recommendations import discretize_rating
from functions.item_based_recommendations import load_ratings
from functions.item_based_recommendations import create_LSH_index
from functions.item_based_recommendations import get_neighbors
from functions.item_based_recommendations import make_recommendations_using_item_based_technique
from functions.item_based_recommendations import evaluate_recommendations_using_decision_support_methods
from functions.item_based_recommendations import evaluate_recommendations_using_nDCG
from functions.item_based_recommendations import evaluate_recommendations_using_MRR
from functions.item_based_recommendations import evaluate_recommendations_using_AP

from functions.matrix_factorization_recommendations import make_recommendations_using_matrix_factorization
from functions.matrix_factorization_recommendations import evaluate_recommendations_by_matrix_factorization

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### *Data* <a class='anchor' id='data'></a>

- *The dataset that will be used is the [Jester Dataset for Recommender Systems and Collaborative Filtering Research](https://eigentaste.berkeley.edu/dataset/)*
- *The dataset contains ratings for 150 jokes from over 50K users*
- *You can see and rate some jokes by clicking [here](https://eigentaste.berkeley.edu/)*
- *You can find more information about Jester by clicking [here](https://eigentaste.berkeley.edu/about.html)*

##### *Read ratings data*

In [3]:
# read ratings data
df_ratings = pd.read_excel('./data/dataset_3_ratings.xls', header=None, usecols=list(range(1,151)))

# preprocess the dataset
df_ratings = preprocess_ratings_dataset(df_ratings)

# shape
print(f'df_ratings.shape: {df_ratings.shape}')

# preview
df_ratings.head()

df_ratings.shape: (54905, 150)


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,0.21875,NaN,-9.28125,-9.28125,NaN,NaN,NaN,NaN,-6.78125,NaN,0.87500,-9.65625,-9.03125,-7.46875,-8.71875,-9.15625,-7.18750,-8.78125,-8.53125,-7.90625,-7.46875,0.03125,8.78125,NaN,8.78125,NaN,8.78125,8.78125,NaN,-0.25000,8.78125,8.78125,NaN,NaN,NaN,NaN,NaN,0.0625,NaN,NaN,NaN,NaN,NaN,NaN,0.06250,9.90625,0.0625,0.0625,8.78125,8.6875,NaN,NaN,NaN,NaN,NaN,NaN,0.03125,0.21875,NaN,NaN,8.78125,8.68750,NaN,-0.12500,8.6875,NaN,NaN,8.78125,NaN,NaN,NaN,9.3125,NaN,NaN,NaN,0.0625,0.12500,NaN,8.78125,NaN,NaN,NaN,8.0000,NaN,9.81250,NaN,8.78125,3.62500,9.3125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.75000,-5.0,2.9375,2.00,-0.15625,2.03125,5.6875,9.65625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,8.78125,8.78125,8.78125,NaN,8.78125,NaN,NaN,NaN,8.6875,0.0,NaN,NaN,NaN,NaN,NaN,3.34375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,-9.68750,NaN,9.93750,9.53125,NaN,NaN,NaN,NaN,9.93750,NaN,0.40625,3.71875,9.65625,-2.68750,-9.56250,-9.12500,9.84375,NaN,NaN,NaN,NaN,9.93750,9.78125,9.81250,9.90625,3.12500,5.50000,-4.25000,NaN,NaN,5.12500,9.84375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.28125,4.96875,9.93750,NaN,NaN,8.00000,-4.3125,NaN,NaN,NaN,NaN,NaN,NaN,-0.78125,4.75000,NaN,NaN,3.68750,-2.12500,NaN,NaN,-5.0000,NaN,NaN,4.78125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.31250,NaN,3.93750,NaN,NaN,NaN,NaN,NaN,9.56250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,-9.84375,NaN,-9.84375,-7.21875,NaN,NaN,NaN,NaN,-2.03125,NaN,-9.93750,-9.96875,-9.87500,-9.81250,-9.78125,-6.84375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.8125,-9.78125,NaN,NaN,-9.8125,0.0625,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.34375,NaN,NaN,1.25,NaN,NaN,NaN,-9.81250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,6.90625,NaN,4.75000,-5.90625,NaN,NaN,NaN,NaN,-0.40625,NaN,-4.03125,3.87500,6.21875,5.65625,6.09375,5.40625,6.37500,7.03125,0.09375,NaN,0.90625,4.18750,0.43750,2.28125,1.18750,0.28125,3.21875,4.90625,NaN,0.53125,3.81250,4.46875,NaN,2.8125,4.8125,0.25,4.4375,5.5625,6.09375,0.5,3.96875,5.78125,6.25000,0.34375,4.28125,1.96875,NaN,1.0625,2.59375,1.1875,0.78125,2.75,NaN,NaN,-0.25,1.8125,1.68750,0.62500,0.15625,NaN,4.90625,0.46875,NaN,0.78125,0.5625,0.40625,NaN,8.00000,3.5625,NaN,NaN,7.0625,0.875,4.8125,NaN,-6.4375,3.90625,4.15625,2.87500,-0.03125,0.46875,0.34375,3.1875,6.28125,6.00000,-0.25,3.56250,6.15625,4.3750,2.5,5.75,7.71875,3.90625,0.84375,0.46875,0.59375,-7.90625,3.81250,NaN,1.4375,NaN,NaN,NaN,NaN,0.31250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,-0.03125,NaN,-9.09375,-0.40625,NaN,NaN,NaN,NaN,7.50000,NaN,-7.21875,-9.43750,0.12500,-9.15625,3.65625,-9.43750,7.90625,NaN,NaN,NaN,NaN,NaN,-9.06250,NaN,6.06250,NaN,5.

##### *Read jokes data*

In [4]:
# read jokes data
df_jokes = pd.read_excel('./data/dataset_3_jokes.xlsx', header=None, names=['joke'])

# preprocess the dataset
df_jokes = preprocess_jokes_dataset(df_jokes)

# shape
print(f'df_jokes.shape: {df_jokes.shape}')

# preview
df_jokes.head()

df_jokes.shape: (150, 2)


,joke_id,joke
0,1,"A man visits the doctor. The doctor says ""I ha..."
1,2,This couple had an excellent relationship goin...
2,3,Q. What's 200 feet long and has 4 teeth? A. ...
3,4,Q. What's the difference between a man and a t...
4,5,Q.\tWhat's O. J. Simpson's Internet address? ...


### *Data Preprocessing* <a class='anchor' id='data_preprocessing'></a>

##### *Unpivot ratings data*

In [5]:
# unpivot the dataset
df_ratings_up = unpivot_ratings(df_ratings)

# shape
print(f'df_ratings_up.shape: {df_ratings_up.shape}')

# preview
df_ratings_up.head()

df_ratings_up.shape: (1842370, 3)


,user_id,joke_id,rating
0,1,5,0.21875
1,1,80,0.06250
2,1,19,-8.71875
3,1,76,9.31250
4,1,127,8.68750


---

## Recommendations Using Item-Based Technique <a class='anchor' id='item_based_technique'></a>

### *Loading User Ratings* <a class='anchor' id='loading_data'></a>

- *In this step, we will preprocess the `df_ratings_up` dataframe*
- *In particular, we will transform its data into a dictionary format*
- *The keys of the dictionary will correspond to each user ID, while the values will be arrays containing the movie IDs rated along with their polarity*

##### *Preprocess `df_ratings_up`*

In [6]:
ratings = load_ratings(df_ratings_up)

### *Create LSH Indices* <a class='anchor' id='lsh_indices'></a>

##### *Create an index for each entity (e.g. joke ID) using Locality Sensitive Hashing (LSH)*

In [7]:
index, hashes = create_LSH_index(ratings)

 50 out of 140 entities indexed.
100 out of 140 entities indexed.
140 out of 140 entities indexed.


### *Make Recommendations* <a class='anchor' id='item_based_recommendations'></a>

##### *Select a random user to recommend jokes*

In [8]:
user_id = 100 # select a user to recommend jokes

##### *Make recommendations*

In [9]:
# recommend jokes for the user provided
to_recommend, already_rated = make_recommendations_using_item_based_technique(user_id,
                                                                              df_ratings_up,
                                                                              df_jokes,
                                                                              ratings,
                                                                              index,
                                                                              hashes)

##### *View recommendations*

In [10]:
# loop through
# joke ID, (joke, polarity, score)
for jid,(j,p,s) in to_recommend.items():
    print(f'JokeID: {jid} - Scaled Votes Score: {round(s,7)}')
    print('='*43)
    print(f'{j}')
    print()

JokeID: 148 - Scaled Votes Score: 3.8155886
Recently a teacher, a garbage collector, and a lawyer wound up together at the Pearly Gates. St. Peter informed them that in order to get into Heaven, they would each have to answer one question. St. Peter addressed the teacher and asked, "What was the name of the ship that crashed into the iceberg? They just made a movie about it." The teacher answered quickly, "That would be the Titanic." St. Peter let him through the gate. St. Peter turned to the garbage man and, figuring Heaven didn't really need all the odors that this guy would bring with him, decided to make the question a little harder: "How many people died on the ship?" Fortunately for him, the trash man had just seen the movie. "1,228," he answered. "That's right! You may enter." St. Peter turned to the lawyer: "Name them."

JokeID: 134 - Scaled Votes Score: 3.5929973
An artist asked the gallery owner if there had been any interest in his paintings currently on display. "I've got g

### *Evaluate Recommendations Using Decision Support Methods* <a class='anchor' id='decision_support_methods'></a>

- *Decision support metrics help to understand how much RecSys are useful in recommending the "right" items to users*
- *In particular, they assist users to take better decision by choosing good items and avoiding bad items*
- *Two of the most commonly used metrics are Precision and Recall*

### *Precision* <a class='anchor' id='precision'></a>

- *Precision is the number of selected items that are relevant*
- *Suppose our RecSys selects 3 items to recommend to users out of which 2 are relevant, then precision will be 66%*
- *Precision is about retrieving the best items to the user assuming that there are more useful items available then you want*

### *Recall* <a class='anchor' id='recall'></a>

- *The recall is the number of relevant items that are selected*
- *Suppose there are 6 relevant items out of which recommender selects 2 relevant items, then recall will be 33%*
- *The recall is about not missing useful items*

<br>

<div style="text-align:center">
    <img src="./images/decision_support_methods.png" alt="description of image">
</div>

##### *Compute Precision and Recall*

In [11]:
evaluate_recommendations_using_decision_support_methods(user_id, df_ratings_up, already_rated)

Precision: 30% - (20/67)
   Recall: 67% - (20/30)


### *Evaluate Recommendations Using Ranking Based Methods* <a class='anchor' id='ranking_based_methods'></a>

<p style='text-align: justify;'><i>Methods we touched so far allow us to understand the overall performance of the results we get from the RecSys. But they provide no information on how the items were ordered. A model can have a good <b>Precision</b> or <b>Recall</b>, but if the top three items that it recommends are not relevant to the user, then the recommendation is not much useful. If the user has to scroll down to search for relevant items then what’s the point of recommendations in the first place? Even without the recommendation user can scroll to look for items of their liking. <b>Ranking based evaluation methods</b> assist us in understanding how suggested items are ordered in terms of their relevancy for the users. They help us to measure quality items ranking.</i></p>

### *nDCG* <a class='anchor' id='ndcg'></a>

<p style='text-align: justify;'><i>nDCG has three parts. First is <b>"CG"</b> which stands for <b>Cumulative Gains</b>. It deals with the fact that most relevant items are more useful than somewhat relevant items that are more useful than irrelevant items. It sums the items based on its relevancy, hence, the term cumulative. Suppose we are asked to score the items based on their relevancy as:</i></p>

- *(P) Most relevant score = 2*
- *(A) Somewhat relevant score = 1*
- *(N) Least relevant score = 0*

*If we are to sum these score we will get cumulative gain for the given items as follows:*

$$ \mathrm{CG_{p}} = \sum_{i=1}^{p} rel_i $$

| Items Ranking | Relevancy Score |
| :-----------: | --------------- |
| Joke 1 | 1 |
| Joke 3 | 2 |
| Joke 2 | 2 |
| Joke 5 | 0 |
| Joke 4 | 1 |
| **CG =** | **6** |

<p style='text-align: justify;'><i>But CG doesn’t account for the position of the items on the list. And, hence, changing the item's position won’t change the CG. This is where the second part of nDCG comes in to play i.e. "D".</i></p>

<p style='text-align: justify;'><i><b>Discounted Cumulative Gain</b>, <b>DCG</b> for short, penalizes the items that appear lower in the list. A relevant item appearing at the end of the list is a result of a bad recommender system and hence that item should be discounted to indicate the bad performance of the model. To do so we divide the relevance score of items with the log of its rank on the list.</i></p>

$$ \mathrm{DCG_{p}} = \sum_{i=1}^{p} \frac{rel_{i}}{\log_{2}(i+1)} $$

| Items Ranking | Relevancy Score |
| :-----------: | --------------- |
| Joke 1 | 1 |
| Joke 3 | 2 |
| Joke 2 | 2 |
| Joke 5 | 0 |
| Joke 4 | 1 |
| **CG =** | **6** |
| **DCG =** | **12.1** |

<p style='text-align: justify;'><i>DCG helps with the ranking, but suppose we are comparing the different lists of the recommender. DCG for each of the lists will be different depending upon where the recommender places the items. What will be DCG for when the most relevant item was placed at 10th position on 20 items list of recommender verses DCG for when the somewhat relevant item was paced at 10th position on 11th item list. To normalize this, "n" of nDCG, the third part, comes in to play.</i></p>

<p style='text-align: justify;'><i><b>nDCG</b> normalized the DCG values of the different number of the items lists. To do so we sort the item list by relevancy and calculate the DCG for that list. This will be the perfect DCG score as items are sorted by their relevancy score. We divide all DCG score of all the list we get by this perfect DCG to get the normalized score for that list. <b>nDCG</b> score ranges from 0 to 1, where 1 indicates perfect ranking of relevant items and 0 indicates no relevant items in the recommendation list.</i></p>

$$ {\mathrm {nDCG_{{p}}}}={\frac {DCG_{{p}}}{IDCG_{{p}}}}={\frac{12.1}{13.9}}=0.87 $$

*where <b>IDCG</b> is ideal discounted cumulative gain,*

$$ \mathrm{IDCG_p} = \sum_{i=1}^{|REL_p|} \frac{rel_i}{\log_2(i+1)} $$

*and ${|REL_p|}$ represents the list of relevant documents (ordered by their relevance) in the corpus up to position p.*

| Perfect Ranking | Relevancy Score |
| :-----------: | --------------- |
| Joke 3 | 2 |
| Joke 2 | 2 |
| Joke 1 | 1 |
| Joke 4 | 1 |
| Joke 5 | 0 |
| **CG =** | **6** |
| **IDCG =** | **13.9** |

##### *Compute nDCG*

In [12]:
evaluate_recommendations_using_nDCG(already_rated)

nDCG: 0.89


### *Mean Reciprocal Rank* <a class='anchor' id='mrr'></a>

<p style='text-align: justify;'><i>Mean Reciprocal Rank, <b>MRR</b> for short, focuses on where is the first relevant item in the recommended list. <b>MRR</b> for a list with the first relevant item at its third position will be greater than for a list with the first relevant item at 4th position. <b>MRR</b> takes the reciprocal of the relevant items’ position and sums them. If relevant items are on positions 2, 3 and 5 on an item list, <b>MRR</b> will be $\frac{1/2 + 1/3 + 1/5}{3}$.</i></p>

| Items Ranking | Relevant Items | Reciprocal Ranking |
| :-----------: | :------------: | :----------------: |
| Joke 1 | No | 0 |
| Joke 3 | Yes | 1/2 |
| Joke 2 | Yes | 1/3 |
| Joke 5 | No | 0 |
| Joke 4 | Yes | 1/5 |

$$ \mathrm{AP} = {\frac{\frac{1}{2} + \frac{1}{3} + \frac{1}{5}}{3}} = 0.34 $$

<p style='text-align: justify;'><i>Typically, a higher <b>MRR</b> score indicates better performance, where a score of 1.0 represents perfect accuracy (i.e., all relevant items are ranked first), while a score of 0.0 represents random guessing. In general, an <b>MRR</b> score above 0.2 is considered to be a good score, but it can vary depending on the specific task and dataset.</i></p>

<p style='text-align: justify;'><i>It's important to keep in mind that the <b>MRR</b> score is just one evaluation metric for a recommender system, and it should be considered alongside other metrics such as precision and recall. It's also important to consider the specific requirements of the application and the preferences of the users to determine what constitutes good performance.</i></p>

##### *Compute MRR*

In [13]:
evaluate_recommendations_using_MRR(already_rated)

Mean Reciprocal Rank: 0.07


### *Average Precision* <a class='anchor' id='ap'></a>

<p style='text-align: justify;'><i>Precision helps to understand the overall performance of the model but doesn’t tell if the items were ranked properly. <b>Average Precision</b>, AP for short, helps to measure the quality of the selected item’s ranking of the recommender model. It calculates the precision for only the relevant items that are recommended.</i></p>

<p style='text-align: justify;'><i>Suppose our model recommends 8 items, as depicted below, out of which 4 are correct and 4 are incorrect. We take the first relevant item and calculate its precision which in our case is the first item, therefore, its precision will be 1/1. Next, calculate precision for the second relevant item (item 3). Its precision will be 2/3. 2 because from 1st till the current item there are two correctly predicted items out of total 3 items. We will do the same for all the relevant items. Lastly, take the mean of the precision list to compute <b>AP</b>. The overall precision for this example is $0.5$, while the <b>AP</b> is $0.75$. Lower <b>AP</b> indicates the quality ranking. <b>AP</b> takes values from 0 (if there are no relevant recommendations) to 1 (if all recommendations are relevant).</i></p>

<p style='text-align: justify;'><i><b>AP</b> gives more weight to the precision of the top recommendations, while precision gives equal weight to all the recommendations. This means that if the relevant items are mainly concentrated at the top of the recommendation list, <b>AP</b> will be higher than precision. On the other hand, if the relevant items are distributed randomly throughout the recommendation list, <b>AP</b> may be lower than precision. Therefore, it's possible for the AP to be either higher or lower than the precision, depending on the order in which the recommendations are presented and the distribution of the relevant items within the list.</i></p>

<br>

<div style="text-align:center">
    <img src="./images/average_precision.png" alt="description of image">
</div>

$$ \mathrm{AP} = {\frac{\frac{1}{1} + \frac{2}{3} + \frac{3}{4} + \frac{4}{7}}{4}} = 0.75 $$

##### *Compute AP*

In [14]:
evaluate_recommendations_using_AP(already_rated)

Average Precision: 37%


---

## Recommendations Using Matrix Factorization <a class='anchor' id='matrix_factorization'></a>

### *Loading Data Using `Surprise` Library* <a class='anchor' id='loading_data_with_surprise'></a>

##### *Load preprocessed ratings data*

In [15]:
# load Reader class
reader = Reader(rating_scale=(-10, 10))

# load ratings dataset with Dataset class
data = Dataset.load_from_df(df_ratings_up[['user_id', 'joke_id', 'rating']], reader)

### *Selecting Number of Factors* <a class='anchor' id='n_factors'></a>

In [16]:
# start time
st = time.time()

# define the hyperparameters to search
param_grid = {'n_factors': range(50,250,50)}

# perform grid search with cross-validation
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5)
grid_search.fit(data)

# get the best number of factors
n_factors = grid_search.best_params['rmse']['n_factors']

# end time
et = time.time()

# print
print(f'     Best RMSE score: {grid_search.best_score["rmse"]}')
print(f'Best hyperparameters: {grid_search.best_params["rmse"]}')
print()
print(f'Elapsed time: {round(et-st)} secs.')

     Best RMSE score: 4.071266526583293
Best hyperparameters: {'n_factors': 200}

Elapsed time: 281 secs.


### *Train SVD Algorithm* <a class='anchor' id='svd'></a>

In [17]:
# define the train set
train_set = data.build_full_trainset()

# initialize the SVD algo
svd = SVD(n_factors=n_factors)

# fit the SVD
svd.fit(train_set)

### *Make Recommendations* <a class='anchor' id='svd_recommendations'></a>

##### *Select a random user to recommend jokes*

In [18]:
user_id = 100 # select a user to recommend jokes

##### *Make recommendations*

In [19]:
recommendations = make_recommendations_using_matrix_factorization(svd, user_id, df_ratings_up, df_jokes)

##### *View recommendations*

In [20]:
for i in range(10):
    print(f'JokeID: {recommendations.iloc[i,0]} - Predicted Rating: {round(recommendations.iloc[i,2],1)}')
    print('===================================')
    print(f'{recommendations.iloc[i,1]}')
    print()

JokeID: 143 - Predicted Rating: 6.0
A preist, a 12-year-old kid, and the smartest guy in the world are on a plane. The pilot screams, "The plane is going down! You have to jump!" He then grabs a parachute and jumps off, leaving only two more parachutes on the plane. The smartest guy in the world says, "I have to go. I mean, I'm the smartest guy in the world!" He grabs a parachute, and jumps. The priest then looks at the 12-year-old kid, and says, "Go, my son. You have a long life to live." The kid calmly responds: "Dude, chill. We'll be fine. The 'smartest guy in the world' took my backpack."

JokeID: 148 - Predicted Rating: 5.9
Recently a teacher, a garbage collector, and a lawyer wound up together at the Pearly Gates. St. Peter informed them that in order to get into Heaven, they would each have to answer one question. St. Peter addressed the teacher and asked, "What was the name of the ship that crashed into the iceberg? They just made a movie about it." The teacher answered quickly

### *Evaluate Recommendations* <a class='anchor' id='svd_evaluation'></a>

##### *Compute RMSE*

In [21]:
evaluate_recommendations_by_matrix_factorization(svd, user_id, df_ratings_up, df_jokes)

RMSE: 0.34


---

*Thank you!*

---